In [74]:
#working

In [1]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
import random
import time
import os
from langgraph.types import Command
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent


In [2]:
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")



In [3]:

# Keep the original simple state - just add what's needed
class GraphState(TypedDict):
    document_content: str
    summary_status: str
    insights_status: str
    summary_output: str
    insights_output: str
    iteration: int
    error_message: str
    current_reasoning: str
    next : str

In [4]:

# Simple but intelligent decision model
class SupervisorDecision(BaseModel):
    """AI Supervisor decision with intelligent reasoning"""
    next_action: Literal[
        "call_both_parallel", 
        "call_summary_only", 
        "call_insights_only", 
        "end_workflow"
    ] = Field(description="What to do next in the workflow")
    
    reasoning: str = Field(description="Why this decision makes sense for the workflow goal")
    
    confidence: float = Field(
        description="Confidence in this decision (0.0 to 1.0)", 
        ge=0.0, le=1.0
    )

In [5]:
from typing import Dict, Any
def intelligent_supervisor(state: GraphState) ->  Dict[str, Any]:
    """
    Smart supervisor that understands the workflow goal:
    - Get meeting minutes summary 
    - Get key insights in parallel
    - Be intelligent about when to retry vs when to finish
    """
    print(f"\n🧠 AI Supervisor thinking... (Iteration: {state['iteration']})")
    
    # Enhanced ReAct-style system prompt
    system_prompt = """
    You are an intelligent workflow supervisor using ReAct (Reasoning + Acting) methodology. 

    ## YOUR MISSION:
    Ensure we successfully get:
    1. A good SUMMARY of the meeting minutes
    2. KEY INSIGHTS from the meeting minutes
    3. Both delivered EFFICIENTLY

    ## ReAct PROCESS - Think step by step:

    **THOUGHT**: First, analyze the current workflow state. What's working? What failed? Why might it have failed?

    **OBSERVATION**: What do you observe about the current status? Look at:
    - What agents have succeeded/failed
    - How many iterations we've done  
    - Whether failures seem temporary (network/API issues) or persistent
    - If we have partial success that might be sufficient

    **ACTION**: Based on your thought and observation, decide the smartest next action for our meeting processing goal.

    ## CURRENT WORKFLOW STATE:
    - Meeting Document: "{doc_preview}"
    - Summary Status: {summary_status}
    - Insights Status: {insights_status} 
    - Current Iteration: {iteration}
    - Last Error: {error_msg}

    ## AVAILABLE ACTIONS:
    - call_both_parallel: Run both agents simultaneously (efficient for fresh start or when both need work)
    - call_summary_only: Focus only on getting the meeting summary
    - call_insights_only: Focus only on getting meeting insights  
    - end_workflow: We have achieved our goal (both summary + insights ready)

    ## INTELLIGENT DECISION GUIDELINES:
    - Iteration 1: Usually start with parallel execution for efficiency
    - One success, one failure: Target retry the failed agent only
    - Both failed early iterations: Likely temporary issues, retry both
    - Multiple failures: Consider if we should accept partial results or continue
    - Both successful: Mission accomplished!

    **Remember**: You're optimizing for getting useful meeting analysis (summary + insights), not perfect success rates.

    Use the ReAct process: THOUGHT → OBSERVATION → ACTION with clear reasoning.
    """
    
    # Format the prompt with current state
    doc_preview = state['document_content'][:100] + "..." if len(state['document_content']) > 100 else state['document_content']
    
    formatted_prompt = system_prompt.format(
        doc_preview=doc_preview,
        summary_status=state['summary_status'],
        insights_status=state['insights_status'], 
        iteration=state['iteration'],
        error_msg=state['error_message'] or "None"
    )
    
    # Setup Groq API with Qwen model
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")
    
    os.environ["GROQ_API_KEY"] = groq_api_key
    
    llm = ChatGroq(
        temperature=0,
        model="deepseek-r1-distill-llama-70b"  
    )
    
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Use ReAct methodology: THOUGHT → OBSERVATION → ACTION. Analyze the workflow state and decide what to do next for our meeting processing goal. Think step by step.")
    ]
    
    # Get structured decision from LLM
    try:
        decision = llm.with_structured_output(SupervisorDecision).invoke(messages)
    except Exception as e:
        print(f"⚠️ LLM call failed: {e}")
        # Fallback to simulation if LLM fails
        decision = simulate_smart_decision(state)
    
    # # Simulate intelligent decision (replace with real LLM)
    # decision = simulate_smart_decision(state)
    
    print(f"💭 AI Reasoning: {decision.reasoning}")
    print(f"⚡ Decision: {decision.next_action} (confidence: {decision.confidence:.2f})")
    
    # # Update state with AI reasoning
    # new_state = state.copy()
    # new_state['next'] = decision.next_action  # Store the next action
    # new_state['current_reasoning'] = decision.reasoning
    
    # return decision.next_action if decision.next_action != "end_workflow" else END
    goto = decision.next_action if decision.next_action != "end_workflow" else END

    print('goto:',goto)

    return {
        "current_reasoning": decision.reasoning,
        "next": decision.next_action # Store the determined next action
    }

    # return Command(
    #         # goto=goto if goto != "FINISH" else "__end__",
    #         goto = goto,
    #         update={
    #             "next": goto,
                
    #             "current_reasoning": decision.reasoning,
    #             # "messages": updated_messages  # Use updated messages
    #         }
    #     ) # pyright: ignore[reportReturnType]
        

In [6]:


def simulate_smart_decision(state: GraphState) -> SupervisorDecision:
    """
    Simulate what an intelligent model like Qwen would decide
    This is just simulation - replace with actual LLM call
    """
    
    summary_status = state['summary_status']
    insights_status = state['insights_status']
    iteration = state['iteration']
    
    # Smart decision making that a good reasoning model would do
    
    # Goal achieved - both successful
    if summary_status == "success" and insights_status == "success":
        return SupervisorDecision(
            next_action="end_workflow",
            reasoning="Perfect! Both summary and insights are ready. Our workflow goal is complete - we have meeting summary + key insights as requested.",
            confidence=1.0
        )
    
    # First iteration - start efficiently  
    if iteration == 1 and summary_status == "pending" and insights_status == "pending":
        return SupervisorDecision(
            next_action="call_both_parallel", 
            reasoning="First attempt - running both summary and insights agents in parallel for efficiency. This is the optimal starting strategy.",
            confidence=0.9
        )
    
    # One succeeded, one failed - targeted retry
    if summary_status == "success" and insights_status == "failed":
        if iteration <= 3:  # Smart about retry limits
            return SupervisorDecision(
                next_action="call_insights_only",
                reasoning="Summary is ready, but insights failed. Retrying only insights agent since summary is already successful. Efficient targeted approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Summary is ready and we've tried insights multiple times. Sometimes partial success is acceptable for meeting processing workflow.",
                confidence=0.6
            )
    
    if insights_status == "success" and summary_status == "failed":
        if iteration <= 3:
            return SupervisorDecision(
                next_action="call_summary_only", 
                reasoning="Insights are ready, but summary failed. Retrying only summary agent since insights are already successful. Focused retry approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Insights are ready and we've tried summary multiple times. We have key insights from the meeting which provides value.",
                confidence=0.6
            )
    
    # Both failed - intelligent retry decision
    if summary_status == "failed" and insights_status == "failed":
        if iteration <= 2:
            return SupervisorDecision(
                next_action="call_both_parallel",
                reasoning="Both agents failed, but it's early in the process. Likely a temporary issue (network/API). Retrying both in parallel - efficient recovery approach.",
                confidence=0.7
            )
        elif iteration <= 4:
            return SupervisorDecision(
                next_action="call_both_parallel", 
                reasoning="Multiple failures but still within reasonable retry range. The meeting document seems valid, so this might be temporary service issues. One more parallel attempt.",
                confidence=0.5
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="After multiple attempts, continuing may not be productive. This could be a deeper issue with the document format or service availability. Ending workflow.",
                confidence=0.8
            )
    
    # Default intelligent fallback
    return SupervisorDecision(
        next_action="call_both_parallel",
        reasoning="Current state requires both agents to run. Taking parallel approach for efficiency in meeting processing workflow.", 
        confidence=0.6
    )


In [7]:
def route_supervisor_decision(state: GraphState) -> str:
    """
    Routes the workflow based on the decision stored by the intelligent supervisor.
    """
    return state['next'] 

In [8]:

# # Keep your original agent functions - they're fine
# def run_summary_agent(state: GraphState) -> GraphState:
#     """Summary agent - focused on meeting summary"""
#     print(f"\n📝 Summary Agent working on meeting minutes...")
#     new_state = state.copy()
    
#     # Simulate work with some intelligence about content
#     success_rate = 0.1 if "meeting" in state['document_content'].lower() else 0.3
    
#     if random.random() < success_rate:
#         new_state['summary_status'] = "success"
#         new_state['summary_output'] = f"Meeting Summary: Key decisions and action items extracted from meeting minutes (iteration {state['iteration']})"
#         print("✅ Summary: Generated meeting summary successfully")
#     else:
#         new_state['summary_status'] = "failed" 
#         new_state['summary_output'] = "Summary generation failed"
#         new_state['error_message'] = "Temporary API issue during summary generation"
#         print("❌ Summary: Failed (likely temporary issue)")
    
#     time.sleep(1)
#     return new_state



In [9]:

def run_summary_agent(state: GraphState) -> dict:
    """
    An agentic node that intelligently generates a meeting summary.
    It has a specific persona, instructions, and robust error handling.
    """
    print("\n🤖 Agentic Summary Node Called...")

    # 1. Define the Agent's Persona and Mission via a System Prompt
    system_prompt = """
    You are an expert Meeting Summarization Agent. Your sole mission is to create a concise, structured, and insightful summary from the provided meeting minutes.

    ## Your Guidelines:
    1.  **Identify Core Content**: Focus on extracting the most critical information:
        - **Key Decisions Made**: What was formally decided?
        - **Action Items**: What are the specific next steps? Who is responsible (owner)? What are the deadlines?
        - **Major Topics Discussed**: Briefly mention the main subjects of conversation.
        - **Outcomes & Resolutions**: What was the final result of the discussions?

    2.  **Prioritize Significance**: Do not just list topics in order. Your value is in identifying what truly matters. A brief 2-minute decision that sets the company's direction is more important than a 30-minute unresolved debate.

    3.  **Structure the Output**: Present the summary in a clean, professional format using Markdown. Use headings (#), subheadings (##), and bullet points (-) for clarity.

    4.  **Be Objective**: Summarize what was said and decided without adding your own opinions or interpretations. Stick to the facts presented in the document.

    Your final output should be ONLY the structured summary, ready to be shared with meeting attendees.
    """

    # 2. Setup the Agent
    # For this task, the agent doesn't need external tools. Its "tool" is its own
    # reasoning capability applied to the text. We use the ReAct agent structure
    # for its robust reasoning loop, even with an empty tool list.
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "Please generate a summary for the following meeting minutes:\n\n---\n\n{{document_content}}")
    ])

    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")

    # The agent uses the LLM and prompt, with no external tools needed for this task.
    summary_agent = create_react_agent(llm, tools=[], prompt=prompt)

    # 3. Invoke the Agent with Error Handling
    try:
        print("🧠 Agent is thinking and generating the summary...")
        # The input to the agent is the document content from our graph state
        agent_input = {"meeting_minutes": state['document_content']}
        result = summary_agent.invoke(agent_input)

        # print("result:", result['messages'][-1].content)

        # The agent's final answer is in the last message of the result
        generated_summary = result['messages'][-1].content
        print("✅ Summary Agent: Generated summary successfully.")
        
        # Return a dictionary to update the graph's state
        return {
            "summary_status": "success",
            "summary_output": generated_summary,
            "error_message": ""  # Clear any previous error message
        }

    except Exception as e:
        print(f"❌ Error in Agentic Summary Node: {e}")
        
        # If the agent fails, report the failure back to the graph state
        # The supervisor can then decide how to handle this failure.
        return {
            "summary_status": "failed",
            "summary_output": "Summary generation failed due to an internal agent error.",
            "error_message": f"Agent Error: {str(e)}"
        }

In [10]:

# def run_insights_agent(state: GraphState) -> dict:
#     """
#     Insights agent - focused on meeting insights.
#     FIX: Now returns a dictionary and clears the error message on success.
#     """
#     print(f"\n🔍 Insights Agent extracting key insights...")
    
#     # Simulate work
#     success_rate = 0.6 if "meeting" in state['document_content'].lower() else 0.3
#     time.sleep(1)

#     if random.random() < success_rate:
#         print("✅ Insights: Extracted meeting insights successfully")
#         # On success, return a success status AND an empty error message
#         return {
#             "insights_status": "success",
#             "insights_output": f"Meeting Insights: Key themes and decisions identified (iteration {state['iteration']})",
#             "error_message": ""
#         }
#     else:
#         print("❌ Insights: Failed (simulating temporary issue)")
#         # On failure, return a failed status AND a descriptive error message
#         return {
#             "insights_status": "failed",
#             "insights_output": "Insights generation failed",
#             "error_message": "Temporary processing issue during insights extraction"
#         }

In [26]:
def run_insights_agent(state: GraphState) -> dict:
    """
    An agentic node that intelligently extracts key insights from meeting minutes.
    It has a specific persona, instructions, and robust error handling.
    """
    print("\n🤖 Agentic Insights Node Called...")

    # 1. Define the Agent's Persona and Mission via a System Prompt
    system_prompt = """
    You are an expert Meeting Insights Agent. Your mission is to extract not just facts, but the underlying meaning and implications from the provided meeting minutes.

    ## Your Guidelines:
    1. **Identify Key Themes**: Go beyond topics. What are the recurring ideas, concerns, or strategic directions? (e.g., "A recurring theme was the concern over budget constraints affecting timelines.")
    2. **Highlight Critical Decisions & Implications**: State the decision and then explain *why it matters*. (e.g., "Decision: Approved 15% marketing budget increase. Implication: This signals a strategic shift towards aggressive market capture for the new product.")
    3. **Surface Actionable Insights**: What can the team learn or do differently based on the discussion? These are not the same as action items. (e.g., "Insight: The debate on resource allocation suggests a lack of clarity in departmental priorities, which should be addressed.")
    4. **Structure the Output**: Present insights in a clear, professional format using Markdown. Use headings and bullet points.

    Your final output should be ONLY the structured insights, ready for strategic review.
    """

    # 2. Setup the Agent
    # TODO: The variable in the prompt must match the key used in the 'invoke' call.
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "Please extract key insights from the following meeting minutes:\n\n---\n\n{{document_content}}")
    ])

    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    llm = ChatGroq(temperature=0.1, model="llama3-70b-8192")

    # The agent uses the LLM and prompt, with no external tools needed for this task.
    insights_agent = create_react_agent(llm, tools=[], prompt=prompt)

    # 3. Invoke the Agent with Error Handling
    try:
        print("🧠 Agent is thinking and extracting insights...")
        agent_input = {"meeting_minutes": state['document_content']}
        result = insights_agent.invoke(agent_input)

        # The agent's final answer is in the last message of the result
        generated_insights = result['messages'][-1].content
        print("✅ Insights Agent: Extracted insights successfully.")
        
        # Return a dictionary to update the graph's state
        return {
            "insights_status": "success",
            "insights_output": generated_insights,
            "error_message": ""  # Clear any previous error message
        }

    except (KeyError, IndexError, Exception) as e:
        print(f"❌ Error in Agentic Insights Node: {e}")
        
        # If the agent fails, report the failure back to the graph state
        return {
            "insights_status": "failed",
            "insights_output": "Insights generation failed due to an internal agent error.",
            "error_message": f"Agent Error: {str(e)}"
        }


In [27]:
from langchain_core.runnables import RunnableParallel

def run_both_parallel_agents(state: GraphState) -> dict:
    """
    Runs summary and insights agents in parallel.
    This now calls both of the new agentic nodes.
    """
    print("\n---RUNNING BOTH AGENTIC NODES IN PARALLEL---")

    parallel_runnable = RunnableParallel(
        summary_result=run_summary_agent,
        insights_result=run_insights_agent 
    )

    parallel_results = parallel_runnable.invoke(state)

    summary_updates = parallel_results['summary_result']
    insights_updates = parallel_results['insights_result']

    combined_updates = {**summary_updates, **insights_updates}

    summary_error = summary_updates.get("error_message", "")
    insights_error = insights_updates.get("error_message", "")

    if summary_error and insights_error:
        combined_updates["error_message"] = f"Summary Error: {summary_error} | Insights Error: {insights_error}"
    elif summary_error:
        combined_updates["error_message"] = summary_error
    elif insights_error:
        combined_updates["error_message"] = insights_error
    else:
        combined_updates["error_message"] = ""

    return combined_updates

In [28]:

# def run_both_parallel_agents(state: GraphState) -> dict:
#     """
#     Runs summary and insights agents in parallel.
#     FIX: Now correctly combines results and manages the error message.
#     """
#     print("\n---RUNNING BOTH AGENTS IN PARALLEL---")

#     # Define the parallel execution. We use the corrected insights agent.
#     # Note: run_summary_agent_agentic is your more robust agent.
#     parallel_runnable = RunnableParallel(
#         summary_result=run_summary_agent,
#         insights_result=run_insights_agent
#     )

#     # Execute in parallel
#     parallel_results = parallel_runnable.invoke(state)

#     summary_updates = parallel_results['summary_result']
#     insights_updates = parallel_results['insights_result']

#     # Combine the updates from both parallel runs
#     combined_updates = {**summary_updates, **insights_updates}

#     # --- Logic to handle combined error messages ---
#     # If both succeeded, the error message from both will be "", which is fine.
#     # If one fails, we want to preserve its specific error message.
#     summary_error = summary_updates.get("error_message", "")
#     insights_error = insights_updates.get("error_message", "")

#     if summary_error and insights_error:
#         combined_updates["error_message"] = f"Summary Error: {summary_error} | Insights Error: {insights_error}"
#     elif summary_error:
#         combined_updates["error_message"] = summary_error
#     elif insights_error:
#         combined_updates["error_message"] = insights_error
#     else:
#         combined_updates["error_message"] = "" # Both succeeded, clear error.

#     return combined_updates



In [29]:
def increment_iteration(state: GraphState) -> GraphState:
    """Track iterations for intelligent decision making"""
    new_state = state.copy()
    new_state['iteration'] += 1
    print(f"\n--- Workflow Iteration: {new_state['iteration']} ---")
    return new_state

In [30]:
# Build the workflow graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("increment_iteration", increment_iteration)
workflow.add_node("intelligent_supervisor", intelligent_supervisor)
workflow.add_node("run_summary_agent", run_summary_agent)
workflow.add_node("run_insights_agent", run_insights_agent)
workflow.add_node("run_parallel_agents", run_both_parallel_agents)

In [31]:

# Set entry point
workflow.set_entry_point("increment_iteration")

# Define initial edge from entry point to supervisor
workflow.add_edge("increment_iteration", "intelligent_supervisor")

In [32]:
# The AI supervisor makes intelligent routing decisions
# - The edge starts from the 'intelligent_supervisor' node.
# - The 'route_supervisor_decision' function defines how to choose the next path.
workflow.add_conditional_edges(
    "intelligent_supervisor",     # Conditional edges originate from the 'intelligent_supervisor' node
    route_supervisor_decision,    # This function defines the routing logic (returns the key for the dict)
    {
        "call_both_parallel": "run_parallel_agents", # Map decision to the parallel execution node
        "call_insights_only": "run_insights_agent",  # Map decision to the insights agent node
        "call_summary_only": "run_summary_agent",    # Map decision to the summary agent node
        "end_workflow": END                          # Map decision to the END state
    }
)


In [33]:
# After each agent completes, return to the supervisor for the next intelligent decision
workflow.add_edge("run_summary_agent", "intelligent_supervisor")
workflow.add_edge("run_insights_agent", "intelligent_supervisor")
workflow.add_edge("run_parallel_agents", "intelligent_supervisor") # Add this edge for parallel execution


In [34]:
# Compile the workflow
app = workflow.compile()


In [35]:

# Test the intelligent workflow
print("🚀 Starting Intelligent Meeting Processing Workflow")

initial_state = GraphState(
        document_content="Meeting Minutes - July 21, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix Update: Bob confirmed the new server is deployed. Alice raised a concern about the budget overrun. It was decided to review the Q3 budget next week. Action Item: Charlie to schedule a budget review meeting by Friday, July 25th.\n2. Marketing Campaign: Discussed the new 'Summer Sale' campaign. It will launch on August 1st. All assets are ready.",
        summary_status="pending",
        insights_status="pending",
        summary_output="",
        insights_output="",
        iteration=0,
        error_message="",
        current_reasoning="",
        next=""
    )

# Run and see the AI make intelligent decisions
for step in app.stream(initial_state):
    if '__end__' not in step:
        print(f"\n📊 Current State: {step}")

print("\n🎯 Workflow completed with AI supervision!")

🚀 Starting Intelligent Meeting Processing Workflow

--- Workflow Iteration: 1 ---

📊 Current State: {'increment_iteration': {'document_content': "Meeting Minutes - July 21, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix Update: Bob confirmed the new server is deployed. Alice raised a concern about the budget overrun. It was decided to review the Q3 budget next week. Action Item: Charlie to schedule a budget review meeting by Friday, July 25th.\n2. Marketing Campaign: Discussed the new 'Summer Sale' campaign. It will launch on August 1st. All assets are ready.", 'summary_status': 'pending', 'insights_status': 'pending', 'summary_output': '', 'insights_output': '', 'iteration': 1, 'error_message': '', 'current_reasoning': '', 'next': ''}}

🧠 AI Supervisor thinking... (Iteration: 1)
💭 AI Reasoning: Since it's the first iteration and both summary and insights are pending, running both agents in parallel will efficiently obtain both results simultaneously.
⚡ Decision: call_both_pa

In [ ]:
if __name__ == '__main__':
#     # This is a simple test to show how the node works in isolation
    test_state = GraphState(
        document_content="Meeting Minutes - July 21, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix Update: Bob confirmed the new server is deployed. Alice raised a concern about the budget overrun. It was decided to review the Q3 budget next week. Action Item: Charlie to schedule a budget review meeting by Friday, July 25th.\n2. Marketing Campaign: Discussed the new 'Summer Sale' campaign. It will launch on August 1st. All assets are ready.",
        summary_status="pending",
        insights_status="pending",
        summary_output="",
        insights_output="",
        iteration=0,
        error_message="",
        current_reasoning="",
        next=""
    )

    result_update = run_summary_agent_agentic(test_state)
    print("\n--- TEST RESULT ---")
    import json
    print(json.dumps(result_update, indent=2))